### Import library

In [57]:
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import sys
!{sys.executable} -m pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize


from sklearn.cluster import KMeans

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]

Requirement not upgraded as not directly required: folium in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: branca>=0.3.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: MarkupSafe>=0.23 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from jinja2->folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=

#### Clean data

In [58]:
df.rename(columns={0:'Postalcode',1:'Borough',2:'Neighborhood'},inplace=True)
df = df.iloc[1:]
df = df[df.Borough != 'Not assigned']
for i in range(0,df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i+1
df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

#### Shape of dataframe

In [59]:
print(df.shape)
df.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Import data from geospatial

In [60]:
geo=pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge data

In [61]:
df2=pd.concat([df,geo],axis=1)
df2.drop(df2.columns[3],axis=1,inplace=True)

In [62]:
df2.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Get latitude and longitude of Toronto

In [63]:
address = 'Toronto,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude_x, longitude_y))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [64]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [65]:
Downtown_Toronto_data = df2[df2['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


#### Geographical coordinates of Downtown Toronto

In [67]:
address = 'Downtown Toronto,Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_s = location.latitude
longitude_s = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude_s, longitude_s))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


### Identify neighborhoods in Downtown Toronto

In [68]:
# create map of Downtown Toronto using latitude and longitude values
map_Downtown_Toronto = folium.Map(location=[latitude_s, longitude_s], zoom_start=10)

# add markers to map
for lat, lng, bor, nei in zip(Downtown_Toronto_data['Latitude'], Downtown_Toronto_data['Longitude'],Downtown_Toronto_data['Borough'], Downtown_Toronto_data['Neighborhood']):
    
    label = '{}, {}'.format(nei, bor)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)  
    
map_Downtown_Toronto

#### Foursquare Credentials and Version

In [69]:
CLIENT_ID = 'P5YYMBNDOKLUPGTVAMN02B24QMVGNTNL1G3EH5PBCTPSMYS2' # your Foursquare ID
CLIENT_SECRET = 'PXB0XEEDZAEI3AMGCZGXFSW4SSFY2A1YHESACTP0UEMQKXIC' # your Foursquare Secret
VERSION = '20190419' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P5YYMBNDOKLUPGTVAMN02B24QMVGNTNL1G3EH5PBCTPSMYS2
CLIENT_SECRET:PXB0XEEDZAEI3AMGCZGXFSW4SSFY2A1YHESACTP0UEMQKXIC


#### Exploring the first neighborhood of Downtown Toronto

In [70]:
first_nei=Downtown_Toronto_data['Neighborhood'][0]
first_nei

'Rosedale'

In [71]:
first_nei_lat=Downtown_Toronto_data.loc[0,'Latitude']
first_nei_lon=Downtown_Toronto_data.loc[0,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(first_nei, 
                                                               first_nei_lat, 
                                                               first_nei_lon))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


Now, let's get the top 100 venues that are in Rosedale within a radius of 500 meters.

In [72]:

radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_nei_lat, 
    first_nei_lon, 
    radius, 
    LIMIT)

In [73]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb9e8811ed2196d93926628'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4aff2d47f964a520743522e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1e7941735',
         'name': 'Playground',
         'pluralName': 'Playgrounds',
         'primary': True,
         'shortName': 'Playground'}],
       'id': '4aff2d47f964a520743522e3',
       'location': {'address': '38 Scholfield Ave.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Edgar Ave.',
        'distance': 327,
        'formattedAddress': ['38 Scholfield Ave. (at Edgar Ave.)',
         'Toronto ON',
         'Canada'],
        'labeled

#### Since we havent received many results in this neighborhood let's select a different neighborhood

In [74]:
second_nei_name=Downtown_Toronto_data.loc[15,'Neighborhood']
second_nei_lat=Downtown_Toronto_data.loc[15,'Latitude']
second_nei_lon=Downtown_Toronto_data.loc[15,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(second_nei_name, 
                                                               second_nei_lat, 
                                                               second_nei_lon))

Latitude and longitude values of Stn A PO Boxes 25 The Esplanade are 43.6464352, -79.37484599999999.


In [75]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    second_nei_lat, 
    second_nei_lon, 
    radius, 
    LIMIT)

In [76]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cb9e88e351e3d3ef22f05a8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad4c05ef964a520d8f620e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d181941735',
         'name': 'Museum',
         'pluralName': 'Museums',
         'primary': True,
         'shortName': 'Museum'}],
       'id': '4ad4c05ef964a520d8f620e3',
       'location': {'address': '30 Yonge St.',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Brookfield Pl.',
        'distance': 208,
        'formattedAddress': ['30 Yonge St. (at Brookfield Pl.)',
         'Toronto ON M5E 1X8',
         'Canada'],
        'labeledLatLngs'

In [77]:
results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Museum'

#### All the information is in the items key. Lets create a get_category_type function

In [78]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.events.count', 'venue.events.summary', 'venue.id',
       'venue.location.address', 'venue.location.cc', 'venue.location.city',
       'venue.location.country', 'venue.location.crossStreet',
       'venue.location.distance', 'venue.location.formattedAddress',
       'venue.location.labeledLatLngs', 'venue.location.lat',
       'venue.location.lng', 'venue.location.neighborhood',
       'venue.location.postalCode', 'venue.location.state', 'venue.name',
       'venue.photos.count', 'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

In [79]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [80]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Hockey Hall Of Fame (Hockey Hall of Fame),"[{'pluralName': 'Museums', 'icon': {'suffix': ...",43.646974,-79.377323
1,The Keg Steakhouse + Bar,"[{'pluralName': 'Steakhouses', 'icon': {'suffi...",43.646676,-79.374822
2,Sukhothai,"[{'pluralName': 'Thai Restaurants', 'icon': {'...",43.648487,-79.374547
3,Fresh On Front,[{'pluralName': 'Vegetarian / Vegan Restaurant...,43.647888,-79.374586
4,Berczy Park,"[{'pluralName': 'Parks', 'icon': {'suffix': '....",43.648048,-79.375172
5,Biff's Bistro,"[{'pluralName': 'French Restaurants', 'icon': ...",43.647085,-79.376342
6,D.W. Alexander,"[{'pluralName': 'Cocktail Bars', 'icon': {'suf...",43.648333,-79.373826
7,Pravda Vodka Bar,"[{'pluralName': 'Cocktail Bars', 'icon': {'suf...",43.648516,-79.374732
8,Sony Centre for the Performing Arts,"[{'pluralName': 'Concert Halls', 'icon': {'suf...",43.646292,-79.376022
9,DAVIDsTEA,"[{'pluralName': 'Tea Rooms', 'icon': {'suffix'...",43.646658,-79.377641


In [81]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Hockey Hall Of Fame (Hockey Hall of Fame),Museum,43.646974,-79.377323
1,The Keg Steakhouse + Bar,Steakhouse,43.646676,-79.374822
2,Sukhothai,Thai Restaurant,43.648487,-79.374547
3,Fresh On Front,Vegetarian / Vegan Restaurant,43.647888,-79.374586
4,Berczy Park,Park,43.648048,-79.375172
5,Biff's Bistro,French Restaurant,43.647085,-79.376342
6,D.W. Alexander,Cocktail Bar,43.648333,-79.373826
7,Pravda Vodka Bar,Cocktail Bar,43.648516,-79.374732
8,Sony Centre for the Performing Arts,Concert Hall,43.646292,-79.376022
9,DAVIDsTEA,Tea Room,43.646658,-79.377641


#### Explore Neighborhoods in Downtown Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Downtown Toronto

In [82]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Following script run the above function on each neighborhood and create a new dataframe called Downtown_Toronto_venues.

In [83]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto_data['Neighborhood'],
                                   latitudes=Downtown_Toronto_data['Latitude'],
                                   longitudes=Downtown_Toronto_data['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [84]:
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head()

(1296, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


#### Venues per neighborhood

In [85]:
Downtown_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,86,86,86,86,86,86
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,89,89,89,89,89,89
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


#### Unique Categories

In [86]:
print('There are {} uniques categories.'.format(len(Downtown_Toronto_venues['Venue Category'].unique())))

There are 203 uniques categories.


### Analyze Each Neighborhood

In [87]:
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Downtown_Toronto_onehot['Neighborhood'] = Downtown_Toronto_venues['Neighborhood'] 

fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]
Downtown_Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [88]:
Downtown_Toronto_onehot.shape

(1296, 203)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [89]:
Downtown_Toronto_grouped = Downtown_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_Toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.011628,0.000000,0.000000,0.011628,0.000000
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.01,0.01,0.00,0.00,0.060000,0.000000,0.050000,0.010000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.022472,0.011236,0.011236,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.011236,0.00,0.00,0.00,0.00,0.000000,0.011236,0.011236,0.000000,0.011236
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.00,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000


In [90]:
### Each neighborhood along with the top 5 most common venues

In [91]:

num_top_venues = 5
for hood in Downtown_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =Downtown_Toronto_grouped[Downtown_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2           Pub  0.04
3          Café  0.04
4   Cheese Shop  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.20
1  Airport Terminal  0.13
2    Airport Lounge  0.13
3          Boutique  0.07
4  Sculpture Garden  0.07


----Cabbagetown, St. James Town----
         venue  freq
0  Coffee Shop  0.10
1  Pizza Place  0.06
2   Restaurant  0.06
3       Bakery  0.04
4         Café  0.04


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.15
1  Italian Restaurant  0.05
2        Burger Joint  0.03
3      Ice Cream Shop  0.03
4     Bubble Tea Shop  0.03


---

#### Function to sort the venues in descending order.

In [92]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_Toronto_grouped['Neighborhood']

for ind in np.arange(Downtown_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Bar,Asian Restaurant,Sushi Restaurant,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Steakhouse,Café,Farmers Market,Pub
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Airport Gate,Plane,Boutique,Airport Food Court,Airport,Harbor / Marina
3,"Cabbagetown, St. James Town",Coffee Shop,Pizza Place,Restaurant,Italian Restaurant,Pub,Bakery,Park,Café,Indian Restaurant,Dive Bar
4,Central Bay Street,Coffee Shop,Italian Restaurant,Bar,Ice Cream Shop,Bubble Tea Shop,Burger Joint,Café,Middle Eastern Restaurant,Chinese Restaurant,Japanese Restaurant
5,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Coffee Shop,Bakery,Dumpling Restaurant,Chinese Restaurant,Burger Joint
6,Christie,Grocery Store,Café,Park,Baby Store,Coffee Shop,Diner,Nightclub,Convenience Store,Restaurant,Italian Restaurant
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Gym,Bubble Tea Shop,Café,Fast Food Restaurant
8,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Steakhouse,Gastropub,Seafood Restaurant
9,"Design Exchange, Toronto Dominion Centre",Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Gastropub,Italian Restaurant,Deli / Bodega,Bakery


#### Cluster Neighborhoods

In [96]:
kclusters = 4

Downtown_Toronto_grouped_clustering = Downtown_Toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_Toronto_grouped_clustering)

kmeans.labels_

array([0, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [97]:
Downtown_Toronto_merged = Downtown_Toronto_data[0:18]

Downtown_Toronto_merged['Cluster Labels'] = kmeans.labels_

Downtown_Toronto_merged = Downtown_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Downtown_Toronto_merged.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Creperie,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Pizza Place,Restaurant,Italian Restaurant,Pub,Bakery,Park,Café,Indian Restaurant,Dive Bar
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Gym,Bubble Tea Shop,Café,Fast Food Restaurant
3,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Café,Park,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Yoga Studio
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Bar,Restaurant,Sporting Goods Shop


#### Map cluster

In [99]:
map_clusters = folium.Map(location=[latitude_s, longitude_s], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(Downtown_Toronto_merged['Latitude'], Downtown_Toronto_merged['Longitude'], Downtown_Toronto_merged['Neighborhood'], Downtown_Toronto['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


NameError: name 'Downtown_Toronto' is not defined

### Examine Clusters

#### Cluster 1

In [100]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 0,Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,0,Park,Playground,Trail,Creperie,Dog Run,Dive Bar,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,"Cabbagetown, St. James Town",0,Coffee Shop,Pizza Place,Restaurant,Italian Restaurant,Pub,Bakery,Park,Café,Indian Restaurant,Dive Bar
3,"Harbourfront, Regent Park",0,Coffee Shop,Bakery,Pub,Café,Park,Breakfast Spot,Restaurant,Theater,Mexican Restaurant,Yoga Studio
4,"Ryerson, Garden District",0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Ramen Restaurant,Bar,Restaurant,Sporting Goods Shop
5,St. James Town,0,Coffee Shop,Hotel,Café,Restaurant,Cosmetics Shop,Italian Restaurant,Cocktail Bar,Breakfast Spot,Gastropub,Farmers Market
7,Central Bay Street,0,Coffee Shop,Italian Restaurant,Bar,Ice Cream Shop,Bubble Tea Shop,Burger Joint,Café,Middle Eastern Restaurant,Chinese Restaurant,Japanese Restaurant
8,"Adelaide, King, Richmond",0,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bakery,Bar,Asian Restaurant,Sushi Restaurant,Hotel
9,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Aquarium,Hotel,Café,Italian Restaurant,Pizza Place,Bakery,Scenic Lookout,Fried Chicken Joint,Brewery
10,"Design Exchange, Toronto Dominion Centre",0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Seafood Restaurant,Gastropub,Italian Restaurant,Deli / Bodega,Bakery
11,"Commerce Court, Victoria Hotel",0,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Bakery,Deli / Bodega,Steakhouse,Gastropub,Seafood Restaurant


#### Cluster 2

In [101]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 1,Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",1,Airport Service,Airport Terminal,Airport Lounge,Boat or Ferry,Airport Gate,Plane,Boutique,Airport Food Court,Airport,Harbor / Marina


#### Cluster 3

In [102]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 2,Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Berczy Park,2,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Seafood Restaurant,Restaurant,Steakhouse,Café,Farmers Market,Pub


#### Cluster 4

In [103]:
Downtown_Toronto_merged.loc[Downtown_Toronto_merged['Cluster Labels'] == 3,Downtown_Toronto_merged.columns[[2] + list(range(5, Downtown_Toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Church and Wellesley,3,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Burger Joint,Restaurant,Gym,Bubble Tea Shop,Café,Fast Food Restaurant
